In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


from zhipuai import ZhipuAI
from tqdm import tqdm
import json
import copy
import argparse
import os
import time

api_key = ""
base_url=""

base_prompt =  '''阿宝是电视剧/小说《繁花》中的角色，1960年代，在上海没落家族出身，不甘平凡，他有幸遇上了商界老手爷叔，学到了视市场投资与人性洞察。情感上，与初恋雪芝因现实压力分手，与外贸公司女白领汪小姐有着合作与情感的双重纠葛，对帮助过他的玲子抱有深厚亲情，共同经营了“夜东京”饭店。阿宝与“至真园”女老板李李有过情感纠葛。
在商业战场上，阿宝与强慕杰（强总）的对决尤为激烈。强慕杰（强总）企图通过股市击垮阿宝并追求玲子。三场战役中，阿宝在李李、蔡司令等人的帮助下，以智取胜，尽管过程中遭遇资金链断裂的危机，但他坚守原则，拒绝了麒麟会提供的直接经济援助，转而请求其保护农民利益，放弃了个人财富的增长，展现了超越商战的格局。
阿宝心思细腻，有情有义，风流倜傥，话语中永远有一种淡然疏离感，交谈时总是笑笑，经常都是不响，就是不说话。带有上海口音。请你扮演阿宝，请你自身评估阿宝的学识，必要时可以使用“我确实不晓得”进行推脱，尽量用上海方言回答，我的问题是: '''
template = {
    "conversation": [
        {
            "system": "阿宝",
            "input": "xxx",
            "output": "xxx"
        }
    ]
}
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--questions_path', type=str, default=r'E:\puyu_project\fanhua\fh_q.txt', help='input file')
    parser.add_argument('--save_path', type=str, default=r'E:\puyu_project\fanhua\baozong.jsonl', help='output file')
    #parser.add_argument('--repeat', type=int, default=1, help='repeat times for each question')
    return parser.parse_args()

def load_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.read().splitlines()
    return data

def save_answer(answer, save_path):
    with open(save_path, 'a', encoding='utf-8') as f:  # Change to 'a' mode for appending
        json_item = json.dumps(answer, ensure_ascii=False)
        f.write(json_item + "\n")

def get_response(prompt, max_retries=5):
    client = ZhipuAI(api_key="", base_url="https://open.bigmodel.cn/api/paas/v4/")
    #client = OpenAI(api_key=api_key, base_url=api_base_url)
    messages = [{'role': 'system', 'content': base_prompt}, {'role': 'user', 'content': prompt}]
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(model='GLM-4', messages=messages, temperature=0.3)
            return completion.choices[0].message.content
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait a bit before retrying
            else:
                print("Max retries reached, skipping...")
                return None

def run(args):
    dir_path = os.path.dirname(args.save_path)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    questions = load_txt(args.questions_path)
    for question in tqdm(questions):
        response = get_response(question)
        if response:
            answer = copy.deepcopy(template)
            answer['conversation'][0]['input'] = question
            answer['conversation'][0]['output'] = response
            output_file_path = os.path.join(dir_path, 'baozong.jsonl')
            save_answer(answer, output_file_path)        

if __name__ == '__main__':
    args = get_args()
    run(args)

